# Some Typical Financial Calculations

# Load Packages

The Roots and Distributions packages have been used in the previous notebooks, but StatsBase not. It contains methods for estimating means, variances, ranks, correlations etc. See https://github.com/JuliaStats/StatsBase.jl. 

In [1]:
using StatsBase, Roots, Distributions

include("printmat.jl")            #function for prettier matrix printing

println4Ps (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# CAPM and Return Autocorrelation

The CAPM regression is

$R_{it}^{e}  =\alpha_{i}+\beta_{i}R_{mt}^{e}+\varepsilon_{it}$,

where $R_{mt}^{e}$ is the market excess return. Theory says that $\alpha=0$, which is easily tested.

### Load Data

In [3]:
xx  = readdlm("Data/MyData.csv",',',header=true)        #monthly return data
x   = xx[1]
ym  = x[:,1]                 #yearmonth
Rme = x[:,2]                 #market excess return
Rf  = x[:,3]                 #interest rate
R   = x[:,4]                 #return small growth stocks
Re  = R - Rf                 #excess returns
T   = size(Rme,1)

388

### CAPM regression

In [4]:
x    = [ones(T) Rme]             #regressors
y    = copy(Re)                  #to get standard OLS notation
b    = x\y                       #OLS
u    = y - x*b                   #residuals
covb = inv(x'x)*var(u)           #cov(b), see any textbook
stdb = sqrt.(diag(covb))         #std(b)
R2a  = 1 - var(u)/var(y)

println("     coeff      std     t-stat")
printmat([b stdb b./stdb])
printlnPs("R2: ",R2a)
printlnPs("no. of observations: ",T)

     coeff      std     t-stat
    -0.504     0.304    -1.656
     1.341     0.066    20.427

R2:      0.519
no. of observations:        388


## Return Autocorrelations

That is, the correlation of $R_{t}^{e}$ and $R_{t-s}^{e}$. 

(It is easily shown that the t-stat of an autocorrelation is $\sqrt{T}$ times the autocorrelation.)

In [5]:
plags = 1:5
xCorr = autocor(Re,plags)         #using the StatsBase package

println("Autocorrelations of the excess returns in Re")
println("\n     lag   autocorrr    t-stat")
printmat([plags xCorr sqrt(T)*xCorr])

Autocorrelations of the excess returns in Re

     lag   autocorrr    t-stat
     1.000     0.216     4.253
     2.000     0.002     0.046
     3.000    -0.018    -0.359
     4.000    -0.065    -1.289
     5.000    -0.027    -0.536



# Value at Risk

The next cell first constructs an simple estimate of $\sigma_t^2$ as a backward looking moving average (the RiskMetrics approach)

$\sigma_t^2 = \lambda \sigma_{t-1}^2 + (1-\lambda) (R_{t-1} -\mu)^2$,
where $\mu$ is the average return (for all data).

Then, we calculate the 95% VaR by assuming a $N(\mu,\sigma_t^2)$ distribution

$\textrm{VaR}_{t} = - (\mu-1.64\sigma_t)$.

If the model is correct, then the loss should not exceed $\textrm{VaR}_{t}$ in more than 5% of the months.

In [6]:
μ = mean(Rme)

λ = 0.95                         #weight on old volatility
σ² = fill(var(Rme),T)                 #RiskMetrics approach to estimate variance     
for t = 2:T
  σ²[t] = λ*σ²[t-1] + (1-λ)*(Rme[t-1]-μ)^2    
end

VaR95 = -(μ - 1.64*sqrt.(σ²))      #VaR at 95% level

println("let's plot it")

let's plot it


In [7]:
YearFrac = floor(ym/100) + (mod(ym,100)-1)/12    #eg 1990.92 for Dec 1990

plot1 = plot(YearFrac,VaR95,color=:blue,legend=false)
plot!(xlims=(1978,2012),ylims=(0,11))
title!("1-month Value at Risk (95%) for US equity market")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 1-month Value at Risk (95%) for US equity market 
 
 
 % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 53.4358,156.548 54.8061,159.017 56.1764,160.406 57.5466,158.64 58.9169,164.779 60.2872,168.463 61.6574,171.08 63.0277,176.991 64.398,174.409 65.7682,179.701 
 67.1385,161.4 68.5088,160.697 69.8791,166.385 71.2493,164.244 72.6196,169.734 73.9899,121.429 75.3601,125.633 76.7304,127.112 78.1007,132.39 79.471,130.209 
 80.8412,136.821 82.2115,142.895 83.5818,149.465 84.952,133.603 86.3223,132.708 87.6926,130.984 89.0628,137.911 90.4331,142.474 91.8034,146.877 93.1737,153.362 
 94.5439,157.122 95.9142,162.002 97.2845,151.412 98.6547,138.679 100.025,140.501 101.395,144.817 102.766,146.194 104.136,148.12 105.506,142.153 106.876,146.902 
 108.247,151.487 109.617,152.056 110.987,153.89 112.357,156.202 113.728,131.211 115.098,138.049 116.468,114.732 117.839,118.077 119.209,125.327 120.579,130.17 
 121.949,136.246 123.32,141.658 124.69,137.958 126.06,144.712 127.43,149.5 128.801,150.113 130.171,156.26 131.541,162.533 132.912,163.394 134.282,168.658 
 135.652,172.851 137.022,176.456 138.393,173.504 139.763,179.359 141.133,184.62 142.503,175.868 143.874,181.345 145.244,182.994 146.614,158.211 147.984,163.843 
 149.355,169.14 150.725,173.288 152.095,178.738 153.466,167.454 154.836,173.381 156.206,178.616 157.576,183.559 158.947,182.965 160.317,188.476 161.687,193.412 
 163.057,197.826 164.428,193.613 165.798,195.401 167.168,189.413 168.539,191.652 169.909,197.036 171.279,189.281 172.649,188.753 174.02,192.964 175.39,192.802 
 176.76,198.137 178.13,185.975 179.501,181.236 180.871,161.634 182.241,163.246 183.612,169.278 184.982,170.873 186.352,135.877 187.722,138.798 189.093,145.06 
 190.463,149.486 191.833,155.884 193.203,158.968 194.574,161.795 195.944,165.821 197.314,168.83 198.685,37.6257 200.055,33.5781 201.425,35.727 202.795,42.4649 
 204.166,48.1824 205.536,55.6878 206.906,64.5256 208.276,72.8883 209.647,77.6128 211.017,85.1237 212.387,89.5221 213.757,96.0314 215.128,103.775 216.498,109.444 
 217.868,116.727 219.239,116.412 220.609,121.618 221.979,128.584 223.349,132.199 224.72,137.344 226.09,143.211 227.46,138.387 228.83,144.92 230.201,150.936 
 231.571,152.206 232.941,158.53 234.312,164.648 235.682,150.794 237.052,157.194 238.422,163.051 239.793,163.988 241.163,152.8 242.533,158.373 243.903,163.122 
 245.274,137.657 246.644,132.598 248.014,137.739 249.385,136.488 250.755,142.432 252.125,145.028 253.495,139.763 254.866,145.513 256.236,151.892 257.606,155.658 
 258.976,153.322 260.347,155.9 261.717,161.418 263.087,166.151 264.457,172.013 265.828,170.874 267.198,148.567 268.568,154.725 269.939,160.993 271.309,163.834 
 272.679,169.879 274.049,175.808 275.42,178.972 276.79,181.582 278.16,184.361 279.53,189.892 280.901,195.307 282.271,197.024 283.641,201.994 285.012,207.06 
 286.382,212.036 287.752,215.855 289.122,216.178 290.493,219.154 291.863,223.814 293.233,228.045 294.603,228.517 295.974,232.707 297.344,236.64 298.714,237.899 
 300.085,241.566 301.455,243.303 302.825,242.639 304.195,233.621 305.566,237.951 306.936,242.174 308.306,240.127 309.676,242.161 311.047,241.396 312.417,241.964 
 313.787,245.985 315.158,239.993 316.528,244.142 317.898,247.728 319.268,247.605 320.639,250.316 322.009,253.203 323.379,254.553 324.749,256.288 326.12,255.514 
 327.49,259.282 328.86,259.521 330.23,260.731 331.601,259.019 332.971,262.615 334.341,264.565 335.712,267.898 337.082,271.357 338.452,273.503 339.822,275.274 
 341.193,276.634 342.563,257.765 343.933,258.968 345.303,253.672 346.674,257.437 348.044,246.355 349.414,248.1 350.785,243.515 352.155,247.037 353.525,237.237 
 354.895,236.961 356.266,225.596 357.636,225.281 359.006,212.492 360.376,209.183 361.747,205.518

# Options

## Black-Scholes Option Price

Let $S$ be the the current spot price of an asset and $y$ be the interest rate.

The Black-Scholes formula for a European call option (on that asset) with strike price $K$ and time to expiration $m$ is

$C  =S\Phi\left(  d_{1}\right)  -e^{-ym}K\Phi\left(d_{2}\right)$, where

$d_{1} =\frac{\ln(S/K)+(y+\sigma^{2}/2)m}{\sigma\sqrt{m}} \ \text{ and } \ d_{2}=d_{1}-\sigma\sqrt{m}$ 

and where $\Phi(d)$ denotes the probability of $x\leq d$ when $x$ has an $N(0,1)$ distribution. All variables except the volatility ($\sigma$) are directly observable. 

In [8]:
function Φ(x)
    #Calculates Pr(z<=x) for N(0,1) variable z
    Pr = cdf(Normal(0,1),x)         #Distributions package
  return Pr
end

function OptionBlackSPs(S,K,m,y,σ)
#Calculates Black-Scholes european call option price
  d1 = ( log.(S./K) + (y+1/2*σ.^2)*m ) ./ (σ*sqrt(m))
  d2 = d1 - σ*sqrt(m)
  c  = S.*Φ(d1) - exp(-y*m).*K.*Φ(d2)
  return c
end

OptionBlackSPs (generic function with 1 method)

In [9]:
σ = 0.4
c1 = OptionBlackSPs(10,10,0.5,0.1,σ)
printlnPs("\n","call price according to Black-Scholes: ",c1)

K = linspace(7,13,51)
c = OptionBlackSPs(10,K,0.5,0.1,σ)

println("let's plot it")


call price according to Black-Scholes:      1.358
let's plot it


In [10]:
plot1 = plot(K,c,color=:red,legend=false)
title!("Black-Scholes call option price (for different strike prices)")
xlabel!("strike price")
ylabel!("option price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 7 
 
 
 8 
 
 
 9 
 
 
 10 
 
 
 11 
 
 
 12 
 
 
 13 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 Black-Scholes call option price (for different strike prices) 
 
 
 strike price 
 
 
 option price 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 28.1037,23.3815 39.4629,35.1682 50.822,46.8095 62.1812,58.2973 73.5404,69.6236 84.8996,80.7808 96.2588,91.7615 107.618,102.559 118.977,113.166 130.336,123.578 
 141.696,133.788 153.055,143.792 164.414,153.584 175.773,163.161 187.132,172.519 198.491,181.656 209.851,190.568 221.21,199.253 232.569,207.711 243.928,215.941 
 255.287,223.941 266.647,231.712 278.006,239.255 289.365,246.57 300.724,253.66 312.083,260.525 323.443,267.167 334.802,273.591 346.161,279.797 357.52,285.789 
 368.879,291.572 380.238,297.147 391.598,302.52 402.957,307.693 414.316,312.673 425.675,317.462 437.034,322.065 448.394,326.487 459.753,330.733 471.112,334.807 
 482.471,338.715 493.83,342.46 505.19,346.048 516.549,349.484 527.908,352.773 539.267,355.919 550.626,358.927 561.985,361.802 573.345,364.548 584.704,367.171 
 596.063,369.674 
 "/>

# Implied Volatility

is the $\sigma$ value that makes the Black-Scholes equation give the same option price as observed on the market. It is often interpreted as the "market uncertainty."

The next cell uses the call option price calculated above as being the "market price." The implied volatility should then be the same as what we used above.

In [11]:
                                #simple (crude) way to solve for implied vol
iv = fzero(σ->OptionBlackSPs(10,10,0.5,0.1,σ)-c1,0.00001,5)

printlnPs("Implied volatility: ",iv,", compare with: $σ")

Implied volatility:      0.400, compare with: 0.4


The next few cells instead some data on options on German government bonds. 

In [12]:
#  LIFFE Bunds option data, trade date April 6, 1994
K = [                        #strike prices; Mx1 vector
      92.00;  94.00;  94.50;  95.00;  95.50;  96.00;  96.50;  97.00;
      97.50;  98.00;  98.50;  99.00;  99.50;  100.0;  100.5;  101.0;
     101.5;  102.0;  102.5;  103.0;  103.5 ];
C = [                        #call prices; Mx1 vector
      5.13;    3.25;    2.83;    2.40;    2.00;    1.64;    1.31;    1.02;
      0.770;   0.570;   0.400;   0.280;   0.190;   0.130;  0.0800;  0.0500;
      0.0400;  0.0300;  0.0200;  0.0100;  0.0100 ];
S = 97.05                #spot price
m = 48/365               #time to expiry in years
y = 0.0                  #Interest rate: LIFFE=>no discounting
N = length(K)

21

In [13]:
iv = fill(NaN,N)
for i = 1:N
  iv[i] = fzero(sigma->OptionBlackSPs(S,K[i],m,y,sigma)-C[i],0.00001,5)
end

println("Strike and iv for data: ")
printmat([K iv])

println("let's plot it")

Strike and iv for data: 
    92.000     0.094
    94.000     0.081
    94.500     0.081
    95.000     0.078
    95.500     0.075
    96.000     0.074
    96.500     0.072
    97.000     0.071
    97.500     0.070
    98.000     0.069
    98.500     0.068
    99.000     0.067
    99.500     0.067
   100.000     0.068
   100.500     0.067
   101.000     0.067
   101.500     0.070
   102.000     0.073
   102.500     0.074
   103.000     0.072
   103.500     0.077

let's plot it


In [14]:
plot1 = plot(K,iv,color=:red,legend=false)
title!("Implied volatility, Bunds options April 6, 1994")
xlabel!("strike price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 92 
 
 
 94 
 
 
 96 
 
 
 98 
 
 
 100 
 
 
 102 
 
 
 0.07 
 
 
 0.08 
 
 
 0.09 
 
 
 Implied volatility, Bunds options April 6, 1994 
 
 
 strike price

# Mean-variance Frontier

Given a vector of average returns ($\mu$) and a variance-covariance matrix ($\Sigma$), the mean-variance frontier show the lowest possible portfolio uncertainty for a given expected portfolio return. It is thus the solution to a quadratic minimization problem. The cells below will use the explicit (matrix) formulas for this solution, but we often have to resort to numerical methods when there are portfolio restrictions.

It is typically plotted with the portfolio standard deviation on the horizontal axis and the portfolio expected return on the vertical axis.

In [15]:
μ = [0.115; 0.095; 0.06]        #expected returns
Σ  = [166  34  58;              #covariance matrix
       34  64   4;
       58   4 100]/100^2                  
Rf = 0.03                       #riskfree return (an interest rate) 

println("μ, Σ and Rf: ")
printmat(μ)
printmat(Σ)
printmat(Rf)

μ, Σ and Rf: 
     0.115
     0.095
     0.060

     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010

     0.030



The functions in the next cell use the explicit solution to the minimization problem. They assume there are no portfolio restrictions. 

In [16]:
function MVCalc(mustar,μ,Σ)           #calculates the std of a portfolio on MVF of risky assets
    n    = length(μ)
    oneV = ones(n,1) 
    Σ_1  = inv(Σ)
    A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
    B    = (μ'Σ_1*oneV)[1]
    C    = (oneV'Σ_1*oneV)[1]
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

function MVCalcRf(mustar,μ,Σ,Rf)           #calculates the std of a portfolio on MVF of (Risky,Riskfree)
    n     = length(μ)
    μe    = μ - Rf                         #expected excess returns            
    Σ_1   = inv(Σ)
    w     = (mustar-Rf)/(μe'Σ_1*μe)[1] * Σ_1 *μe
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalcRf (generic function with 1 method)

In [17]:
mustar = linspace(Rf,0.15,101)
L      = length(mustar)
StdRp  = fill(NaN,L)
for i = 1:L
    StdRP_i, = MVCalc(mustar[i],μ,Σ)             #, = means skipping the remaining output
    StdRp[i] = StdRP_i
end    

plot3 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("MVF, only risky assets")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 MVF, only risky assets 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 518.234,300.416 510.019,297.645 501.855,294.875 493.744,292.105 485.691,289.334 477.698,286.564 469.768,283.794 461.905,281.023 454.114,278.253 446.398,275.483 
 438.762,272.712 431.211,269.942 423.75,267.171 416.384,264.401 409.121,261.631 401.965,258.86 394.924,256.09 388.006,253.32 381.217,250.549 374.566,247.779 
 368.062,245.009 361.715,242.238 355.534,239.468 349.529,236.698 343.712,233.927 338.095,231.157 332.688,228.387 327.506,225.616 322.56,222.846 317.864,220.076 
 313.433,217.305 309.279,214.535 305.416,211.765 301.859,208.994 298.62,206.224 295.712,203.454 293.148,200.683 290.938,197.913 289.093,195.143 287.621,192.372 
 286.53,189.602 285.824,186.832 285.508,184.061 285.582,181.291 286.048,178.521 286.901,175.75 288.139,172.98 289.754,170.21 291.74,167.439 294.086,164.669 
 296.783,161.899 299.819,159.128 303.181,156.358 306.857,153.587 310.832,150.817 315.094,148.047 319.628,145.276 324.421,142.506 329.459,139.736 334.728,136.965 
 340.217,134.195 345.912,131.425 351.802,128.654 357.875,125.884 364.121,123.114 370.529,120.343 377.09,117.573 383.795,114.803 390.634,112.032 397.6,109.262 
 404.686,106.492 411.884,103.721 419.187,100.951 426.589,98.1807 434.085,95.4103 441.669,92.64 449.337,89.8696 457.082,87.0993 464.901,84.329 472.79,81.5586 
 480.744,78.7883 488.761,76.0179 496.837,73.2476 504.968,70.4773 513.152,67.7069 521.386,64.9366 529.667,62.1662 537.993,59.3959 546.361,56.6255 554.77,53.8552 
 563.218,51.0849 571.702,48.3145 580.22,45.5442 588.772,42.7738 597.356,40.0035 605.969,37.2332 614.612,34.4628 623.281,31.6925 631.977,28.9221 640.698,26.1518 
 649.443,23.3815 
 "/>

In [18]:
StdRpRf  = fill(NaN,L)
for i = 1:L
    StdRpRf_i, = MVCalcRf(mustar[i],μ,Σ,Rf) 
    StdRpRf[i] = StdRpRf_i
end    

plot4 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("MVF, risky and riskfree assets")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 MVF, risky and riskfree assets 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 518.234,300.416 510.019,297.645 501.855,294.875 493.744,292.105 485.691,289.334 477.698,286.564 469.768,283.794 461.905,281.023 454.114,278.253 446.398,275.483 
 438.762,272.712 431.211,269.942 423.75,267.171 416.384,264.401 409.121,261.631 401.965,258.86 394.924,256.09 388.006,253.32 381.217,250.549 374.566,247.779 
 368.062,245.009 361.715,242.238 355.534,239.468 349.529,236.698 343.712,233.927 338.095,231.157 332.688,228.387 327.506,225.616 322.56,222.846 317.864,220.076 
 313.433,217.305 309.279,214.535 305.416,211.765 301.859,208.994 298.62,206.224 295.712,203.454 293.148,200.683 290.938,197.913 289.093,195.143 287.621,192.372 
 286.53,189.602 285.824,186.832 285.508,184.061 285.582,181.291 286.048,178.521 286.901,175.75 288.139,172.98 289.754,170.21 291.74,167.439 294.086,164.669 
 296.783,161.899 299.819,159.128 303.181,156.358 306.857,153.587 310.832,150.817 315.094,148.047 319.628,145.276 324.421,142.506 329.459,139.736 334.728,136.965 
 340.217,134.195 345.912,131.425 351.802,128.654 357.875,125.884 364.121,123.114 370.529,120.343 377.09,117.573 383.795,114.803 390.634,112.032 397.6,109.262 
 404.686,106.492 411.884,103.721 419.187,100.951 426.589,98.1807 434.085,95.4103 441.669,92.64 449.337,89.8696 457.082,87.0993 464.901,84.329 472.79,81.5586 
 480.744,78.7883 488.761,76.0179 496.837,73.2476 504.968,70.4773 513.152,67.7069 521.386,64.9366 529.667,62.1662 537.993,59.3959 546.361,56.6255 554.77,53.8552 
 563.218,51.0849 571.702,48.3145 580.22,45.5442 588.772,42.7738 597.356,40.0035 605.969,37.2332 614.612,34.4628 623.281,31.6925 631.977,28.9221 640.698,26.1518 
 649.443,23.3815 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 54.7703,300.416 59.4998,297.645 64.2293,294.875 68.9587,292.105 73.6882,289.334 78.4176,286.564 83.1471,283.794 87.8766,281.023 92.606,278.253 97.3355,275.483 
 102.065,272.712 106.794,269.942 111.524,267.171 116.253,264.401 120.983,261.631 125.712,258.86 130.442,256.09 135.171,253.32 139.901,250.549 144.63,247.779 
 149.36,245.009 154.089,242.238 158.818,239.468 163.548,236.698 168.277,233.927 173.007,231.157 177.736,228.387 182.466,225.616 187.195,222.846 191.925,220.076 
 196.654,217.305 201.384,214.535 206.113,211.765 210.843,208.994 215.572,206.224 220.301,203.454 225.031,200.683 229.76,197.913 234.49,195.143 239.219,192.372 
 243.949,189.602 248.678,186.832 253.408,184.061 258.137,181.291 262.867,178.521 267.596,175.75 272.326,172.98 277.055,170.21 281.784,167.439 286.514,164.669 
 291.243,161.899 295.973,159.128 300.702,156.358 305.432,153.587 310.161,150.817 314.891,148.047 319.62,145.276 324.35,142.506 329.079,139.736 333.809,136.965 
 338.538,134.195 343.267,131.425 347.997,128.654 352.726,125.884 357.456,123.114 362.185,120.343 366.915,117.573 371.644,114.803 376.374,112.032 381.103,109.262 
 385.833,106.492 390.562,103.721 395.292,100.951 400.021,98.1807 404.751,95.4103 409.48,92.64 414.209,89.8696 418.939,87.0993 423.668,84.329 428.398,81.5586 
 433.127,78.7883 437.857,76.0179 442.586,73.2476 447.316,70.4773 452.045,67.7069 456.775,64.9366 461.504,62.1662 466.234,59.3959 470.963,56.6255 475.692,53.8552 
 480.422,51.0849 485.151,48.3145 489.881,45.5442 494.61,42.7738 499.34,40.0035 504.069,37.2332 508.799,34.4628 513.528,31.6925 518.258,28.9221 522.987,26.1518 
 527.717,23.3815 
 "/>

# (extra) Mean-variance Frontier without Short Selling

The code below solves (numerically) the following minimization problem 

$\min \text{Var}(R_p) \> \text{ s.t. } \>  \text{E}R_p = \mu^*$,
 
and where we require $w_i\ge 0$
 
The code below calls on
```
quadprog(zeros(N),Sigma,A,'=',b,zeros(N),ones(N),IpoptSolver(print_level=1))
```

This means that we minimize
    $0'w + w' \Sigma w/2$ subject to the restrictions $A w = b$ and $0 \leq w \leq 1$. The equality constraints ($A w = b$) are used to impose $1'w=1$ and $\mu'w = \mu^*$ (se the code below).
    
The packages MathProgBase and Ipopt are documented at https://github.com/JuliaOpt/MathProgBase.jl and https://github.com/JuliaOpt/Ipopt.jl.

In [19]:
using MathProgBase, Ipopt

In [20]:
function MeanVarNoSSPs(mu,Sigma,mustar)
    K = length(mustar)                #MV with no short-sales, numerical minimization
    N = length(mu)
    vv = find((mustar .>= minimum(mu)) & (mustar .<= maximum(mu)))  #solve only if feasible
    lb = zeros(N,1)              #w >= 0
    ub = ones(N,1)               #w <= 1
    A  = [ones(1,N);mu']         #1'w=1, mu'w = mustar
    w_p   = fill(NaN,(K,N))
    StdRp = fill(NaN,K)                       #to store results in
    for i = vv
        b  = [1;mustar[i]]
        Sol = quadprog(zeros(N),Sigma,A,'=',b,zeros(N),ones(N),IpoptSolver(print_level=1))
        w_i = Sol.sol
        if Sol.status == :Optimal
          w_p[i,:] = w_i'
          StdRp[i] = sqrt((w_i'*Sigma*w_i)[1])
        end
    end
    return StdRp,w_p
end

MeanVarNoSSPs (generic function with 1 method)

In [21]:
Std_no_ss, = MeanVarNoSSPs(μ,Σ,mustar)  

plot(StdRp*100,mustar*100,color=:red,linewidth=2,label="Risky only")
plot!(Std_no_ss*100,mustar*100,color=:green,line=(:dash,2),label="Risky only (no ss) ")
plot!(xlims=(0,15),ylims=(0,15))
title!("MVF (portfolio restrictions)")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 MVF (portfolio restrictions) 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 518.234,300.416 510.019,297.645 501.855,294.875 493.744,292.105 485.691,289.334 477.698,286.564 469.768,283.794 461.905,281.023 454.114,278.253 446.398,275.483 
 438.762,272.712 431.211,269.942 423.75,267.171 416.384,264.401 409.121,261.631 401.965,258.86 394.924,256.09 388.006,253.32 381.217,250.549 374.566,247.779 
 368.062,245.009 361.715,242.238 355.534,239.468 349.529,236.698 343.712,233.927 338.095,231.157 332.688,228.387 327.506,225.616 322.56,222.846 317.864,220.076 
 313.433,217.305 309.279,214.535 305.416,211.765 301.859,208.994 298.62,206.224 295.712,203.454 293.148,200.683 290.938,197.913 289.093,195.143 287.621,192.372 
 286.53,189.602 285.824,186.832 285.508,184.061 285.582,181.291 286.048,178.521 286.901,175.75 288.139,172.98 289.754,170.21 291.74,167.439 294.086,164.669 
 296.783,161.899 299.819,159.128 303.181,156.358 306.857,153.587 310.832,150.817 315.094,148.047 319.628,145.276 324.421,142.506 329.459,139.736 334.728,136.965 
 340.217,134.195 345.912,131.425 351.802,128.654 357.875,125.884 364.121,123.114 370.529,120.343 377.09,117.573 383.795,114.803 390.634,112.032 397.6,109.262 
 404.686,106.492 411.884,103.721 419.187,100.951 426.589,98.1807 434.085,95.4103 441.669,92.64 449.337,89.8696 457.082,87.0993 464.901,84.329 472.79,81.5586 
 480.744,78.7883 488.761,76.0179 496.837,73.2476 504.968,70.4773 513.152,67.7069 521.386,64.9366 529.667,62.1662 537.993,59.3959 546.361,56.6255 554.77,53.8552 
 563.218,51.0849 571.702,48.3145 580.22,45.5442 588.772,42.7738 597.356,40.0035 605.969,37.2332 614.612,34.4628 623.281,31.6925 631.977,28.9221 640.698,26.1518 
 649.443,23.3815 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#008100; stroke-width:1.6; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 415.632,231.157 403.895,228.387 392.456,225.616 381.349,222.846 370.607,220.076 360.27,217.305 350.379,214.535 340.981,211.765 332.126,208.994 323.868,206.224 
 316.263,203.454 309.37,200.683 303.248,197.913 297.955,195.143 293.546,192.372 290.072,189.602 287.574,186.832 286.083,184.061 285.619,181.291 286.048,178.521 
 286.901,175.75 288.139,172.98 289.754,170.21 291.74,167.439 294.086,164.669 296.783,161.899 299.819,159.128 303.181,156.358 306.857,153.587 310.832,150.817 
 315.094,148.047 319.628,145.276 324.421,142.506 329.459,139.736 334.728,136.965 340.843,134.195 349.337,131.425 360.09,128.654 372.873,125.884 387.454,123.114 
 403.605,120.343 421.119,117.573 439.812,114.803 459.518,112.032 480.098,109.262 501.431,106.492 
 "/>
 
 
 
 
 Risky only 
 
 
 
 Risky only (no ss)